# Objective: Grouping large data frames by different variables and applying aggregate functions. 

## Two important pandas functions: groupby() and agg().

## The dataset that we are using: phone_data.csv

# Import the libraries

In [102]:
import pandas as pd
import dateutil                     # For Date parsing

# Load the data from the *.csv file

In [103]:

df = pd.read_csv('phone_data.csv')
print(df.head())
print("\n\n", df.shape)                      # (830, 7)

# Note:
# Here 'duration' is in:
#             seconds: for each call, 
#             MB     : for each data entry, 
#             Integer(usually 1): for each sms entry

   index            date  duration  item    month   network network_type
0      0  15/10/14 06:58    34.429  data  2014-11      data         data
1      1  15/10/14 06:58    13.000  call  2014-11  Vodafone       mobile
2      2  15/10/14 14:46    23.000  call  2014-11    Meteor       mobile
3      3  15/10/14 14:48     4.000  call  2014-11     Tesco       mobile
4      4  15/10/14 17:27     4.000  call  2014-11     Tesco       mobile


 (830, 7)


# Check the date column datatype and convert to Datetime if need be...

In [106]:
# Check existing 'date' column type
print("Type of the date column:", type(df.date[0]))        # str

# Convert date from [str] to [datetime]
df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)

# Check the 'date' column again
print("Type of the date column:", type(df.date[0]))        # Timestamp

# Note: Timestamp is the pandas equivalent of python's Datetime and is interchangeable with it in most cases. 

Type of the date column: <class 'str'>
Type of the date column: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


# Summarize the DataFrame

In [107]:
# Fetch all column names
print("All column names:------------------->", df.columns)


# Count the number of rows in the dataset
print("\n\nTotal number of observations:----------------->", df['item'].count())
     

# What was the longest call duration?
print("\n\nLongest call:--------->", df.duration.max())

# Details of the longest call
print("\n\nLongest call detail:------->\n",df.loc[df.duration.idxmax()])
# NOTE: 
# idxmax() returns th-e indexes/ row labels, depending upon the filter criterion. In this case row label/index = 816
# In case the indexes are not not unique, the idxmax() might return multiple rows. 
# To avoid this use the following technique:
df = df.reset_index()         # So that the rows have unique index. 
print("\n\nLongest call detail [After reset index]:------->\n",df.loc[df.duration.idxmax()])
# Reference: https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan


All column names:-------------------> Index(['index', 'date', 'duration', 'item', 'month', 'network',
       'network_type'],
      dtype='object')


Total number of observations:-----------------> 830


Longest call:---------> 10528.0


Longest call detail:------->
 index                           816
date            2015-03-04 12:29:00
duration                      10528
item                           call
month                       2015-03
network                    landline
network_type               landline
Name: 816, dtype: object


Longest call detail [After reset index]:------->
 level_0                         816
index                           816
date            2015-03-04 12:29:00
duration                      10528
item                           call
month                       2015-03
network                    landline
network_type               landline
Name: 816, dtype: object


In [108]:
# How many seconds of total phone calls were made?
print(df.duration[df.item=='call'].sum())     # A more verbose representation would be:

print(df['duration'][df['item'] == 'call'].sum())

92321.0
92321.0


In [109]:
# How many entries [call/data/sms] are there for each month?
df['month'].value_counts()

2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

In [110]:
# Number of non-null unique network entries
print(df.network.nunique())                       # 9

# The unique network entries
print(df.network.unique())                        # ['data','Vodafone','Meteor','Tesco','Three','voicemail','landline',
                                                  #  'special', 'world']
    

9
['data' 'Vodafone' 'Meteor' 'Tesco' 'Three' 'voicemail' 'landline'
 'special' 'world']


# Summary Statistics

In [111]:
df.describe()

,level_0,index,duration
count,830.000000,830.000000,830.000000
mean,414.500000,414.500000,117.804036
std,239.744656,239.744656,444.129560
min,0.000000,0.000000,1.000000
25%,207.250000,207.250000,1.000000
50%,414.500000,414.500000,24.500000
75%,621.750000,621.750000,55.000000
max,829.000000,829.000000,10528.000000


# Summary Statistics-dedicated functions


In [112]:
'''
1. count : Number of non-null observations
2. sum   : Sum of the numeric values
3. mean  : Mean of the values
4. median: Arithmetic median of the values
5. min   : Minimum
6. max   : Maximum
7. mode  : Mode
8. abs   : Absolute Values
9. mad   : mean absolute deviation
10. prod : Product of values
11. std  : Unbiased standard deviation
12. var  : Unbiased variance
13. sem  : Unbiased std error of the mean
14. skew : Unbiased skewness(3rd. Moment)
15. kurt : Unbiased kurtosis(4th. Moment)
16. quantile: sample quantile
17. cumsum : Cumulative Sum
18. cumprod: Cumulative Product
19. cummax : Cumulative Maximum
20. cummin : Cumulative Minimum
'''

'\n1. count : Number of non-null observations\n2. sum   : Sum of the numeric values\n3. mean  : Mean of the values\n4. median: Arithmetic median of the values\n5. min   : Minimum\n6. max   : Maximum\n7. mode  : Mode\n8. abs   : Absolute Values\n9. mad   : mean absolute deviation\n10. prod : Product of values\n11. std  : Unbiased standard deviation\n12. var  : Unbiased variance\n13. sem  : Unbiased std error of the mean\n14. skew : Unbiased skewness(3rd. Moment)\n15. kurt : Unbiased kurtosis(4th. Moment)\n16. quantile: sample quantile\n17. cumsum : Cumulative Sum\n18. cumprod: Cumulative Product\n19. cummax : Cumulative Maximum\n20. cummin : Cumulative Minimum\n'

# Grouping in Pandas DataFrame
### We use the groupby() which returns the "DataFrameGroupBy" object.
### We use the .groups variable which is infact a dictionary, whose keys are the computed unique groups while the corresponding values are the axis labels, belonging to each group.

In [113]:
# Group by month
df.groupby(['month']).groups.keys()


dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [114]:
# Determine the number of records in a specific group
len(df.groupby(['month']).groups['2014-11'])


230

In [115]:
# NOTE: Functions like max(), min(), first(), last(), mean() can be quickly applied to the DataFrameGroupBy Object, 
# to get the summary statistics of the grouped clusters. 

# To get the first entry of each month...
df.groupby(['month']).first()

# Observe that the groupby() sets the index/ row label == Variable of grouping. 

,level_0,index,date,duration,item,network,network_type
month,,,,,,,
2014-11,0,0,2014-10-15 06:58:00,34.429,data,data,data
2014-12,228,228,2014-11-13 06:58:00,34.429,data,data,data
2015-01,381,381,2014-12-13 06:58:00,34.429,data,data,data
2015-02,577,577,2015-01-13 06:58:00,34.429,data,data,data
2015-03,729,729,2015-02-12 20:15:00,69.000,call,landline,landline


In [116]:
# To get the last entry of each month...
df.groupby(['month']).last()


,level_0,index,date,duration,item,network,network_type
month,,,,,,,
2014-11,230,230,2014-11-13 22:31:00,1.000,sms,Vodafone,mobile
2014-12,388,388,2014-12-14 19:54:00,25.000,call,Three,mobile
2015-01,593,593,2015-01-14 23:36:00,1.000,sms,Three,mobile
2015-02,728,728,2015-02-12 06:58:00,34.429,data,data,data
2015-03,829,829,2015-03-14 00:16:00,1.000,sms,world,world


In [117]:
# Get the sum of the durations of each month
df.groupby(['month'])['duration'].sum()                        # The duration is recorded in [seconds]
                                                               # This duration contains calls + sms + data
    

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [118]:
# Get the sum of the durations for calls only for each network provider.
df[df['item'] == 'call'].groupby('network')['duration'].sum()                   # In seconds...


network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [119]:
# Get the number of dates/entries in each month
df.groupby(['month'])['date'].count()                       # The number of dates n each month.


month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

# Complex grouping - on more than 1 variable

In [120]:
# How many calls, sms, data entries in each month
df.groupby(['month', 'item'])['date'].count()


month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [121]:
# How many calls, sms, data entries in each month per network provider
df.groupby(['month', 'network_type'])['date'].count()


month    network_type
2014-11  data             29
         landline          5
         mobile          189
         special           1
         voicemail         6
2014-12  data             30
         landline          7
         mobile          108
         voicemail         8
         world             4
2015-01  data             31
         landline         11
         mobile          160
         voicemail         3
2015-02  data             31
         landline          8
         mobile           90
         special           2
         voicemail         6
2015-03  data             29
         landline         11
         mobile           54
         voicemail         4
         world             3
Name: date, dtype: int64

# GroupBy output format - Series/ DataFrames?
## The output of a groupby and aggregation operation could be a pandas series/ DataFrame. 
## You can determine the specific type by using the type(). 
### Thumbrule: if the specific operation returns more than 1 columns ----> pandas DataFrame
### if the specific aggregation operation returns a single column ----> pandas Series. 

### You can change the datatype of the output, depending upon the way you pass the columns.

In [122]:
# To determine the total duration of usage per month
res = df.groupby(['month'])['duration'].sum()
print(res)
print(type(res))                                              # Returns a pandas Series


month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64
<class 'pandas.core.series.Series'>


In [123]:
# To determine the total duration of usage per month
res = df.groupby(['month'])[['duration']].sum()
print(res)
print(type(res))                                           # Returns a DataFrame


          duration
month             
2014-11  26639.441
2014-12  14641.870
2015-01  18223.299
2015-02  15522.299
2015-03  22750.441
<class 'pandas.core.frame.DataFrame'>


# Group by with no index set on the grouping variable
### by default, the groupby() returns a result whose index is the variable of grouping. 
### We can avoid this by passing an additional parameter called as_index = False. 

In [124]:
df.groupby(['month'], as_index=False)['duration'].sum()

# An alternative format
df.groupby('month', as_index=False).agg({'duration':'sum'})    # Specify the aggregation tasks as a Python dict


,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


# agg() - to perfom multiple statistical computations on group, in a single go...
## The syntax is similar to the MongoDB's aggregation function/ aggregation pipeline
## The aggregation tasks are specified in the form of a Python dictionary. 

In [125]:
# Group the DataFrame by month and item and extract a number of stats from each group.
df.groupby(['month', 'item']).agg({'duration': 'sum',
                                   'network_type': 'count',
                                   'date': 'first'
                                  })
# Here we performed 3 aggregation tasks in one go:
# 1. Find the sum of durations for each group
# 2. Find the number of network type entries per group
# 3. Get the first date per group 


duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [128]:
# Since the aggregation tasks are specified as a Python dictionary, you can very well segregate it outside, of the groupby()
# Also you can define custom logics using anonymous lambda functions
print("Type of the date column:", type(df.date[0]))                  # Timestamp
aggTasks = {'duration': 'sum', 
            'date': lambda x: max(x) 
           }

df.groupby('month').agg(aggTasks)


Type of the date column: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


,duration,date
month,,
2014-11,26639.441,2014-11-13 22:31:00
2014-12,14641.870,2014-12-14 19:54:00
2015-01,18223.299,2015-01-14 23:36:00
2015-02,15522.299,2015-02-12 06:58:00
2015-03,22750.441,2015-03-14 00:16:00


In [129]:
# Another example of complex aggregation:
df.groupby(['month', 'item']).agg({'duration': [min, max, sum], 
                                   'network_type': 'count', 
                                   'date': [min, 'first', 'nunique']
                                  })

# Aggregation tasks acheived:
# 1. Find out the min, max and sum of the duration column
# 2. Find the number of network_type entries. 
# 3. Find the min, first and number of unique dates per group. 

# Observe that the groupby() set mutliple-indexes on the column axis. 
# Working with multi-indices is tricky!

duration                       network_type                date  \
                  min        max        sum        count                 min   
month   item                                                                   
2014-11 call    1.000   1940.000  25547.000          107 2014-10-15 06:58:00   
        data   34.429     34.429    998.441           29 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000           94 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000           79 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870           30 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000           48 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000           88 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299           31 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000           86 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000           67 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299           31 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000           39 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000           47 2015-02-12 20:15:00   
        data   34.429     34.429    998.441           29 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000           25 2015-02-19 18:46:00   

                                          
                           first nunique  
month   item                              
2014-11 call 2014-10-15 06:58:00     104  
        data 2014-10-15 06:58:00      29  
        sms  2014-10-16 22:18:00      79  
2014-12 call 2014-11-14 17:24:00      76  
        data 2014-11-13 06:58:00      30  
        sms  2014-11-14 17:28:00      41  
2015-01 call 2014-12-15 20:03:00      84  
        data 2014-12-13 06:58:00      31  
        sms  2014-12-15 19:56:00      58  
2015-02 call 2015-01-15 10:36:00      67  
        data 2015-01-13 06:58:00      31  
        sms  2015-01-15 12:23:00      27  
2015-03 call 2015-02-12 20:15:00      47  
        data 2015-02-13 06:58:00      29  
        sms  2015-02-19 18:46:00      17

In [145]:
# Eliminate the hassle of multi-index on columns. 
groupRes = df.groupby(['month']).agg({'duration': [min, max, 'mean']})
print(groupRes)
print(groupRes.index)

groupRes.columns = groupRes.columns.droplevel(level = 0)
print(groupRes)
print(groupRes.index)

groupRes.rename(columns = {"min": "min_duration", 
                           "max": "max_duration", 
                           "mean": "mean_duration"}).head()

#print(groupRes.head())

        duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891
Index(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'], dtype='object', name='month')
         min      max        mean
month                            
2014-11  1.0   1940.0  115.823657
2014-12  1.0   2120.0   93.260318
2015-01  1.0   1859.0   88.894141
2015-02  1.0   1863.0  113.301453
2015-03  1.0  10528.0  225.251891
Index(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'], dtype='object', name='month')


,min_duration,max_duration,mean_duration
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


In [1]:
# Reference: https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/